In [11]:
import pandas as pd

# Lecture des datas
#df = pd.read_csv("df_avec_categorie_part3.csv")  # Fichier contenant la partie 2 à traduire
df = pd.read_csv("df_avec_categorie_part3_lang.csv")
#df_test = pd.read_csv("df_p2.1.csv")             # Fichier contenant 10% de la partie 2 (je l'ai scindé de df_p2.1.csv à df_p2.10.csv pour mes tests)
len(df[(df["langue"] == "en") & (df["trad"].isna())])


2405

In [14]:
import deepl

#auth_key = "7f03f88a-3bd9-4fd4-bbd1-c95a5574f3bd:fx"  # Replace with your key
auth_key = "3d4f32b5-9c3b-4ebf-8f7c-3ee448b618ff:fx" # clé backup
translator = deepl.Translator(auth_key)

df = pd.read_csv("df_avec_categorie_part3_lang.csv")
df["trad"] = df[(df["langue"] == "en") & (df["trad"].isna())]["truncated"].apply(lambda text: translator.translate_text(text, target_lang="FR"))
df.to_csv("df_avec_categorie_part3_lang2.csv") 


QuotaExceededException: Quota for this billing period has been exceeded, message: Quota Exceeded

In [60]:
from langdetect import detect, DetectorFactory

DetectorFactory.seed = 42       # On fixe le seed pour avoir toujours le même résultat 
                                # En effet, il peut y avoir des résultats variables et aléatoires sur les textes trop courts entre 2 exécutions 

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

df["langue"] = df["merged"].apply(detect_language)
df['nb_char'] = df["merged"].str.len()

df.to_csv("df_avec_categorie_part3_lang.csv") 

In [75]:
df[df["langue"] == "en"]["nb_char"].describe()

# On compte le nombre de caractères total
df['truncated'] = df['merged'].apply(lambda x: x[:400].rsplit(' ', 1)[0] if len(x) > 400 else x)
df['nb_char'] = df["truncated"].str.len()

df.to_csv("df_avec_categorie_part3_lang.csv") 

In [13]:
df[(df["langue"] == "en") & (df["trad"].isna())]["truncated"].str.len().sum()

350748

In [54]:
# On compte le nombre de caractères total
df['truncated'] = df['merged'].apply(lambda x: x[:400].rsplit(' ', 1)[0] if len(x) > 400 else x)
df['nb_char'] = df["truncated"].str.len()
 
df.to_csv("df_avec_categorie_part3_lang.csv") 

print("Il y a ", df[df["langue"] != "fr"]["nb_char"].sum(),"caractères") 
print("Il y a ", df[df["langue"] == "en"]["nb_char"].sum(),"caractères")
print("Il y a ", df[df["langue"] == "en"]["nb_char"].head(3000).sum(),"caractères")
print(df[df["langue"] == "en"]["nb_char"].describe())


Il y a  847020 caractères
Il y a  685581 caractères
Il y a  497691 caractères
count    4194.000000
mean      163.467096
std       180.644116
min         1.000000
25%        36.250000
50%        60.000000
75%       263.000000
max       499.000000
Name: nb_char, dtype: float64


In [49]:
import spacy

nlp = spacy.load("xx_ent_wiki_sm")

def mask_entities(text):
    doc = nlp(text)
    for ent in doc.ents:
        text = ' '.join(text.replace(ent.text, "").split())  # Remplace les entités par des placeholders
    return text

df_en = df[df["langue"] == "en"]
df_en['description_masquee'] = df_en['description'].apply(mask_entities)


Le casque est idéal pour .


In [39]:
print(df[df["langue"] != "fr"]["nb_char"].describe())

print("il y a", len(df[(df["langue"] == "en") & (df["nb_char"]>200)]),"description non française de plus de 200 caractères")
print("il y a", len(df[(df["langue"] == "en")]),"description non française")

print("Il y a ", df[(df["langue"] == "en") & (df["nb_char"]<=200)]["nb_char"].sum(),"caractères")

count    7486.000000
mean      171.035266
std       376.173882
min         1.000000
25%        30.000000
50%        45.000000
75%        98.750000
max      4100.000000
Name: nb_char, dtype: float64
il y a 1152 description non française de plus de 200 caractères
il y a 4194 description non française
Il y a  176542 caractères


In [ ]:
import spacy

nlp = spacy.load("fr_core_news_lg")  # Modèle NER spécialisé en français

text = "Le casque Sony est idéal pour Call of Duty."
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)  

In [52]:
import spacy
#nlp = spacy.load("xx_ent_wiki_sm")  # Modèle NER multilingue

nlp = spacy.load("en_core_web_sm")
#nlp = spacy.load("fr_core_news_lg")  # Modèle NER spécialisé en français

# ORG (Marques)
# PRODUCT (Produits, jeux vidéo, films)
# WORK_OF_ART (Films, livres)
#PERSON (Si pertinent, pour les créateurs de marque)

entities = ["ORG", "PRODUCT", "WORK_OF_ART", "MISC", "CATEGORY"]
entities = ["ORG"]

org = set()

for text in df_en["truncated"].head(500):
    #text = "Ce téléphone Samsung est inspiré du film The Matrix."
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in entities:
            org.add(ent.text)

display(org)

{'accessoire salle bain',
 'addon microsoft train',
 'albertus seba',
 'alliages daluminiumbr',
 'ann phoenix',
 'atlas corp humain',
 'attache mousse zodiac',
 'austin morris pub',
 'automne',
 'avery zweckform',
 'bcw bgx board',
 'betrayer',
 'birdsong aden',
 'bleu',
 'bleu rouge',
 'blumfeldt',
 'borja',
 'carte catch slam',
 'casa',
 'chicco chaise',
 'cin',
 'classhps',
 'classhpssilverspan span classhps stainless steel',
 'coffret stylo chantal thomas rose stylo',
 'coin bank',
 'congress',
 'conseil clay craft extrudeuse outil dcoration gteau',
 'cozy high sea',
 'cribbage board',
 'cribbage chh inc',
 'designcan',
 'dexxa game pad',
 'dimension w tome',
 'disney rapunzel classic',
 'draco',
 'dragon ball z',
 'edition ab ab',
 'eurographics am dfmirrortetris',
 'fabercastell kit starter',
 'fifa',
 'figaro litteraire',
 'figaro magazine',
 'fimi adescription brand',
 'ford',
 'fpv mm drone',
 'fulton v',
 'homme noir',
 'household food wrap film',
 'housse',
 'ichiban kuji',
